In [1]:
import os
from azureml.core import Model, Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice, AksWebservice
from azureml.pipeline.wrapper import PipelineRun

In [2]:
subscription_id = '4f455bd0-f95a-4b7d-8d08-078611508e0b'
resource_group = 'fundamental'
workspace_name = 'fundamental3'
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
# for loading module
namespace = workspace_name
InteractiveLoginAuthentication(tenant_id=tenant_id)
workspace = Workspace(subscription_id, resource_group, workspace_name)
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy', 'aml-compute-tmp'])


In [3]:
# get a PipelineRun object
experiment_name = "deploy"
experiment = Experiment(workspace, experiment_name)
run_id = '24f0ba77-1ef8-4a9a-b9a8-9ed14d98190e'
pipeline_run = PipelineRun(experiment, run_id)

In [4]:
# get a StepRun object
step_run = pipeline_run.find_step_run(name='FastText Train')[0]

In [5]:
# download model from the output port of the Train module
port = step_run.get_port(name='Trained model dir')
saved_path = port.download(local_path='data', overwrite=True)
print('model save at: {}'.format(saved_path))

Downloaded azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir/BestModel, 1 files out of an estimated total of 1
model save at: data\azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir


In [6]:
# register model for deployment
model_name = os.listdir(saved_path)[0]
model_path = os.path.join(saved_path, model_name)
model = Model.register(workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy': 1})

Registering model model_for_deploy


In [7]:
env_list = Environment.list(workspace)
name = 'env_for_deployment'
if name not in env_list:
    file_path = 'deployment/env_for_deployment.yaml'
    env = Environment.from_conda_specification(name=name, file_path=file_path)
    env = env.register(workspace=workspace)
else:
    env = Environment.get(workspace=workspace, name=name)

In [8]:
# define inference configuration
entry_script = 'scoring_for_deployment.py'
source_directory = 'deployment'
inference_config = InferenceConfig(entry_script=entry_script, source_directory=source_directory,
                                   environment=env)

In [9]:
# deploy locally
service_name = 'local-deploy-test'
models = [model]
port = 8892
deployment_config = LocalWebservice.deploy_configuration(port=port)
service_locally = Model.deploy(workspace=workspace, name=service_name, models=models, inference_config=inference_config,
                               deployment_config=deployment_config)
service_locally.wait_for_deployment(show_output=True)
print(service_locally.state)
print(service_locally.get_logs())

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry fundamental33c005c1f.azurecr.io
Logging into Docker registry fundamental33c005c1f.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM fundamental33c005c1f.azurecr.io/azureml/azureml_ac0df2cc6dbad51226a8d176555b98dc
 ---> 0b06c2e718e3
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 7dd3a759dfb5
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjRmNDU1YmQwLWY5NWEtNGI3ZC04ZDA4LTA3ODYxMTUwOGUwYiIsInJlc291cmNlR3JvdXBOYW1lIjoiZnVuZGFtZW50YWwiLCJhY2NvdW50TmFtZSI6ImZ1bmRhbWVudGFsMyIsIndvcmtzcGFjZUlkIjoiYTdjMmFjYWEtYzhmMS00NDhiLWI4OTQtYzJlN2E3MWIzYTMyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 5413de418930
 ---> 53f56dfba485
Step 4/5 : RUN mv '/var/azureml-app/tmptxntviox.py' /var/azureml-app/main.py
 ---> Running in 8e974225d66a
 ---> d01d34265e88
Step 5/5 : CMD 

In [12]:
# deploy to ACI (Azure Container Instances)
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand.
# to delete the existing service, let overwrite be True
service_name = 'aci-deploy-test'
models = [model]
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service_aci = Model.deploy(workspace, service_name, models=models, inference_config=inference_config,
                           deployment_config=deployment_config, overwrite=True)
service_aci.wait_for_deployment(show_output=True)
print(service_aci.state)

True
Running.......
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [13]:
# deploy to AKS (Azure Kubernetes Service)
# workspace with AKS
aks_subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
aks_resource_group = 'DesignerDRI'
aks_workspace_name = 'DesignerDRI_EASTUS'
aks_workspace = Workspace(aks_subscription_id, aks_resource_group, aks_workspace_name)

In [14]:
# deploy to AKS (Azure Kubernetes Service)
# register model in this workspace
model = Model.register(aks_workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy': 1})

Registering model model_for_deploy


In [9]:
# deploy to AKS (Azure Kubernetes Service)
# load aks_target
name = 'aks-dev'
aks_resource_group = 'myresourcegroup'
cluster_name = 'designerdri-weu1d991123'
# compute_target_name_list = [target.name for target in AksCompute.list(aks_workspace)]
# if not name in compute_target_name_list:
#     attach_config = AksCompute.attach_configuration(resource_group=aks_resource_group, cluster_name=cluster_name)
#     aks_target = ComputeTarget.attach(aks_workspace, name, attach_config)
#     aks_target.wait_for_completion(show_output=True)
# else:
#     aks_target = AksCompute(aks_workspace, name)
aks_target = AksCompute(aks_workspace, name)
aks_target

NameError: name 'aks_workspace' is not defined

In [11]:
# deploy to AKS (Azure Kubernetes Service)
# every time we deploy to AKS, we need to change the service_name or we delete the existing service beforehand
service_name = 'aks-deploy-test9'
models = [model]
token_auth_enabled = True
# Only one type of Auth may be enabled
if token_auth_enabled:
    # key auth
    auth_enabled = False
else:
    auth_enabled = True

deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                       token_auth_enabled=token_auth_enabled,
                                                       auth_enabled=auth_enabled)
service = Model.deploy(aks_workspace, service_name, models, inference_config, deployment_config, aks_target,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
print(service.state)

NameError: name 'aks_workspace' is not defined

In [24]:
# consume
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication

# Get a token to authenticate to the compute instance from remote
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Create and submit a request using the auth header
headers = auth_header
# Add content type header
headers.update({'Content-Type': 'application/json'})
# print(headers)


standard_sample_input = {'param': {'input_sentence': '受疫情影响, 今年很多学生不得不在家上课'}}
standard_sample_input = json.dumps(standard_sample_input)

service = service_locally
service = service_aci

# service = service_aks
# token, refresh_by = service.get_token()
# headers['Authorization']=f'Bearer {token}'


response = requests.post(service.scoring_uri, data=standard_sample_input, headers=headers)
print(service.scoring_uri)
print(response)
# print(response.status_code)
# print(response.elapsed)
print(response.content)
print(response.json())

http://814d582b-1b3e-487a-949d-bc7be11a24db.eastasia.azurecontainer.io/score
<Response [200]>
b'"politics"'
politics
